In [0]:
%sql
SELECT 
  period_month
  ,legal_entity_name
  ,SUM(rev_recognized_amount) AS rev_recognized_amount
  ,SUM(rev_sku_all) AS rev_sku_all
  ,SUM(contract_amortization_value) AS contract_amortization_value
FROM fin_demo.rev.fact_revenue_recognition
GROUP BY 1, legal_entity_name



Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT * FROM fin_demo.legal.dim_all_contracts WHERE agreement_type = 'Outbound' LIMIT 100

In [0]:
%sql
SELECT * FROM fin_demo.rev.stg_revenue_transactions 
WHERE customer_id = 20000005
  AND DATE(transaction_date) = DATE('2024-08-01')
LIMIT 100

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
WITH base AS (
  SELECT customer_id, COUNT(DISTINCT rev_category) AS rev_count 
  FROM fin_demo.rev.stg_revenue_transactions 
  GROUP BY 1
) 
SELECT rev_count, COUNT(customer_id) AS freq
FROM base
GROUP BY 1

In [0]:
%sql
SELECT * FROM fin_demo.rev.fact_revenue_recognition WHERE period_month >= DATE('2023-06-01')
ORDER BY period_month, third_party_id DESC, contract_id
LIMIT 1000

In [0]:
%sql
SELECT
    COALESCE(rev_trx.period_month, commit.amortization_month) AS period_month,
    COALESCE(rev_trx.third_party_id, commit.third_party_id) AS third_party_id,
    COALESCE(rev_trx.customer_name, commit.third_party_name) AS customer_name,
    COALESCE(rev_trx.legal_entity_id, commit.legal_entity_id) AS legal_entity_id,
    COALESCE(rev_trx.legal_entity_name, commit.legal_entity_name) AS legal_entity_name,
    commit.contract_id,
    commit.contract_number,
    commit.monthly_amortized_value,
    rev_trx.period_month,
    rev_trx.period_transaction_count,
    rev_trx.rev_sku_all,
    rev_trx.rev_sku_labor,
    rev_trx.rev_sku_compute,
    rev_trx.rev_sku_materials,
    rev_trx.rev_sku_subcontractor,
    rev_trx.rev_sku_equipment,
    rev_trx.rev_sku_overhead,
    rev_trx.rev_billed_amount,
    rev_trx.rev_received_amount,
    rev_trx.rev_linear_payments,
    rev_trx.rev_nonlinear_payments

FROM fin_demo.rev.stg_commit_revenue AS commit
FULL JOIN (

    SELECT
        contract_id
        ,third_party_id
        ,customer_name
        ,coa_meta.legal_entity_id
        ,coa_meta.legal_entity_name
        ,TRUNC(transaction_date, 'month') AS period_month
        ,COUNT(rev_trx_id) AS period_transaction_count
        ,SUM(amount) AS rev_sku_all
        ,SUM(amount) FILTER (WHERE rev_category = 'labor') AS rev_sku_labor
        ,SUM(amount) FILTER (WHERE rev_category = 'compute') AS rev_sku_compute
        ,SUM(amount) FILTER (WHERE rev_category = 'materials') AS rev_sku_materials
        ,SUM(amount) FILTER (WHERE rev_category = 'subcontractor') AS rev_sku_subcontractor
        ,SUM(amount) FILTER (WHERE rev_category = 'equipment') AS rev_sku_equipment
        ,SUM(amount) FILTER (WHERE rev_category = 'overhead') AS rev_sku_overhead
        ,SUM(billed_amount) AS rev_billed_amount
        ,SUM(billed_amount) FILTER (WHERE payment_received_date IS NOT NULL) AS rev_received_amount
        ,SUM(amount) FILTER (WHERE TRUNC(transaction_date, 'month') = TRUNC(payment_received_date, 'month')) AS rev_linear_payments
        ,SUM(amount) FILTER (WHERE TRUNC(transaction_date, 'month') < TRUNC(payment_received_date, 'month')) AS rev_nonlinear_payments
    FROM fin_demo.rev.stg_revenue_transactions AS rev_trx
    GROUP BY 1, 2, 3, 4, 5, 6

    ) AS rev_trx
    ON commit.contract_id = rev_trx.contract_id
    AND commit.amortization_month = rev_trx.period_month
    AND commit.third_party_id = rev_trx.third_party_id
    AND commit.legal_entity_id = rev_trx.legal_entity_id 

In [0]:
%sql 
WITH base AS (
  SELECT
    DATE(DATE_TRUNC('month', transaction_date)) AS month
    ,customer_id
    ,customer_name
    ,platform
    ,SUM(fab_amount) AS fab_amount
  FROM fin_demo.rev.stg_dbx_consumption_revenue
  GROUP BY 1, 2, 3, 4
) 
SELECT 
  *
  ,IF(
    fab_amount > 
      ((LAG(fab_amount, 1) OVER (PARTITION BY customer_id ORDER BY month)
        + LAG(fab_amount, 2) OVER (PARTITION BY customer_id ORDER BY month)
        + LAG(fab_amount, 3) OVER (PARTITION BY customer_id ORDER BY month)
      )/3)
    ,'increasing' 
    ,'decreasing'
    ) AS trend
FROM base
ORDER BY 3, 1


Databricks visualization. Run in Databricks to view.